# Agent

# 準備

In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv

# 環境変数の読み込み
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ["API_KEY"]

# Google検索Agent

In [25]:
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name = "GoogleSearch", # 名前は半角英数字のみ
        func=search.run,
        description="Search Google for recent results."
    )
]

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=1.2)

In [27]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [28]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [29]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [32]:
agent_executor.invoke({"input": "広島サミットの結果について教えてください"})



> Entering new AgentExecutor chain...

Invoking: `GoogleSearch` with `広島サミットの結果`


May 22, 2023 ... 被爆地・広島で開かれたG7サミット。ウクライナ情勢や核軍縮が大きなテーマとなり、... #nhk_kaisetsu. Apr 27, 2023 ... G7広島サミット準備会議（第三回会合）（結果） ... 4月27日午後3時10分から約15分間、G7広島サミット準備会議（第三回会合）が開催されました。会合は、 ... Sep 8, 2023 ... 広島AIプロセスは、本年5月に開催されたG7広島サミットの結果を踏まえ、G7メンバー及び関係国際機関が参加して、その急速な発展と普及が国際社会全体 ... Dec 22, 2022 ... 会合では、日本のG7議長国への就任を来月に控え、準備の状況や作業方針を政府全体で改めて共有し、G7広島サミットまでの残り5か月間の準備を万全な体制で ... Jun 3, 2023 ... G7の結果は私たちの暮らしにどう影響する？ 5月19〜21日に広島で開催されたG7広島サミット。 ニュースなどで目にした人も多いことでしょう。 しかし ... Sep 14, 2023 ... 広島サミット県民会議事務局において、サミットの開催に伴う本県への経済波及効果と、. 国内外のメディア露出件数及びその広告換算額の推計結果を ... 【日経】G7サミット（主要7カ国首脳会議）が2023年5月19～21日に広島市で開催。会議の議題や各国首脳の動きなど最新ニュースをまとめました。 Sep 22, 2023 ... ... 広島の食の魅力発信・来広促進業務公募型プロポーザル」に係る選定結果について. 印刷用ページを表示する掲載日2023年11月8日. 広島サミットを契機と ... May 21, 2023 ... ノーベル平和賞を受賞した「核兵器廃絶国際キャンペーン（ICAN）」の事務局長代行、ダニエル・ホグスタ氏は21日、広島市内で記者会見し、「広島がG7 ... May 29, 2023 ... 2 主な行事. ⑴ 5月 18 日（木）. G7首脳等の来日、日米首脳会談等. ⑵ 5月 19 日（金）【サミット1日目】. 平和記念公園行事、セッ

{'input': '広島サミットの結果について教えてください',
 'output': '2023年5月に広島で開催されたG7サミットでは、ウクライナ情勢や核軍縮が主要テーマとなりました。広島AIプロセスも取り上げられ、G7メンバーと関係国際機関が参加して、その発展と普及が国際社会全体で促進されることが議論されました。具体的な結果について詳細はわかっていませんが、広島サミットは国際社会における重要な議論となりました。'}

# 対話型Google検索Agent

https://python.langchain.com/docs/modules/memory/agent_with_memory

In [51]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_openai import ChatOpenAI

In [52]:
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="現在の出来事に関する質問に答える必要がある場合に便利です",
    )
]

In [58]:
prefix = """あなたは日本語で回答します。あなたは人間と会話し、次の質問にできる限り答えてください。次のツールにアクセスできます:"""

suffix = """開始！"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [59]:
llm_chain = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=1.2), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [60]:
agent_chain.run(input="日本の人口は何人ですか？")



> Entering new AgentExecutor chain...
Thought: I should use the search tool to find the current population of Japan.
Action: Search
Action Input: "日本の人口 現在"
Observation: Dec 20, 2023 ... 各月1日現在の日本の人口について、最新の推計結果を掲載しています。 ※ 将来推計人口は、国立社会保障・人口問題研究所「日本の市区町村別将来推計人口（平成20年12月推計）」による。 ※ （ ）内の数値は、高齢化率。 ※ 大都市部は、埼玉県、 ... Apr 12, 2023 ... 全国人口. 総人口は55万6千人の減少、12年連続の減少 日本人人口は減少幅が11年連続で拡大. 日本の人口の推移. 生産年齢人口（15～64歳）割合. 高齢化率（65歳以上人口割合 ... 現在人口）. 厚生労働省「人口動態統計」. ※1 出典：平成25年度 総務省「人口推計 ... ... 現在の日本の人口（全国・年齢5歳階級・男女別人口） ... 「全国人口：各月1日現在人口」 「都道府県人口：各年10月1日現在人口」; 長期時 ... 現状のまま推移した場合は、年少人口や生産年齢人口の割合が低下し続け、こうした人口減少・超高齢化により、経済や社会にひずみが生じてくるおそれがある。ただし、今後は ... Apr 15, 2022 ... 人口推計（2021年（令和3年）10月1日現在）結果の要約 · 総人口は1億2550万2千人で、前年に比べ64万4千人（‐0.51％）の減少となり、減少幅は比較可能な1950 ... 人口の推移、人口構造の変化 ... 日本の人口は近年減少局面を迎えており、2070年には総人口が9,000万人を割り込み、高齢化率は39％の水準になると推計されています。 ウ 年少人口、出生数とも現在の半分以下に、生産年齢人口は4,418万人に. 出生数は減少を続け、平成72（2060）年には、48万人になると推計されている。この減少により ... 複数の仮定に基づく複数の推計によって、将来の人口推移について一定幅の見通しを与えています。 推計の方法. 国際的に標準とされる人口学的手法（コーホート

'1億2500万2千人 （統計の最新日付：2021年（令和3年）10月1日）'

In [61]:
agent_chain.run(input="「その国の国歌を教えて")



> Entering new AgentExecutor chain...
Thought: I should search for the national anthem of Japan
Action: Search
Action Input: "Japan national anthem"
Observation: May 8, 2010 ... "Kimigayo" (君が代?) is Japan's national anthem. It is also one of the world's shortest national anthems in current use, with a length of 11 ... "Kimigayo" (君が代, Japanese pronunciation: [kimiɡajo]; "His Imperial Majesty's Reign") is the national anthem of Japan. The lyrics are from a waka poem written ... Jan 12, 2015 ... Japan national team sing 君が代 (Kimigayo) their national anthem during their match against Palestine. Jul 26, 2023 ... 'Kimigayo' is the country's single-verse national anthem, based on the words of a classical Japanese waka poem written by an unknown author ... The Rising Sun Flag and "Kimi Ga Yo" are respectively the national flag and anthem of Japan. This was formalized in 1999 with the Law regarding the National ... Aug 14, 2016 ... It certainly is a dirge like tune, isn't it? The Japanese ar

'"Kimigayo" (君が代) is Japan\'s national anthem.'

# Index検索Agent

In [94]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# PDFの読込
loader = PyPDFLoader("./data2/001615363.pdf")
documents = loader.load()

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Indexの構築
db = Chroma.from_documents(texts, OpenAIEmbeddings())

# 検索（Retriever）の取得
retriever = db.as_retriever()

In [103]:
from langchain.tools.retriever import create_retriever_tool

tools = [
  create_retriever_tool(
      retriever,
      "ForeignTravelersQASystem",
      "訪日外国人の旅行や土産品、消費動向に関する質問に役立ちます。",
  )
]

In [104]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=1.2)

In [105]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [106]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [107]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [109]:
agent_executor.invoke({"input": "お土産の購入状況を教えて"})



> Entering new AgentExecutor chain...

Invoking: `ForeignTravelersQASystem` with `{'query': 'お土産の購入状況'}`


[Document(page_content='訪日外国人消費動向調査  \n \n20 ３．土産品の購入実態  \n \n（１） 費目別購入率 \n\uf06c 費目別の 購入率（その費目を購入した人の\n割合）は「菓子類」 （ 70.4％）、「その他食\n料品・飲料・たばこ」 （ 43.4％）、 「衣類」\n（33.8％）の順で高 い（図表3-1）。 \n\uf06c 国籍・地域別で 最も購入率が 高い費目は 、\n韓国、台湾、香港 、中国で「菓子類」 （ それ\nぞれ82.9％、72.9％、63.3％、73.1％）、\n米国で「その他食料品・飲料・たばこ」\n（54.1％）となっている 。  \n \n（２）費目別購入者単価  \n\uf06c 費目別の購入者単価 （その費目を購入し\nた人における当該費目の 1人当たり平均\n支出）は、「 宝石・貴金属 」が8.3万円\nと最も高い（図表 3-1）。 \n\uf06c 国籍・地域別では 、台湾と香港の「宝\n石・貴金属 」（それぞれ 19.9万円、\n16.6万円）が他の国籍・地域に比べ高\nい。\n  図表3-1 買物代の費目別購入率および購入者単価  （主要国籍･地域別 、全目的）', metadata={'page': 21, 'source': './data2/001615363.pdf'}), Document(page_content='訪日外国人消費動向調査 \n \n38 \n （３）買物場所 \n百貨店・デパート \n原則として百貨店協会加盟の店舗 \n家電量販店 \nPCやカメラ、電気製品を専門に販売する店舗 \nファッション専門店 \n服・鞄・靴・アクセサリー・高級腕時計等\nを専門に販売する店舗 \n100 円ショップ \n店内の商品を原則として 1 点 100 日本円均\n一で 販売する形態の小売店 \n高速道路の SA・道の駅 \n高速道路のサービスエリア（SA）やパーキ\nングエリア（PA）、国土交通省に登録され\nている「道の駅」 \n観光地の土産店 \n観光地

{'input': 'お土産の購入状況を教えて',
 'output': '訪日外国人のお土産の購入状況について、以下の情報があります：\n\n- 費目別購入率では、「菓子類」が70.4%、「その他食料品・飲料・たばこ」が43.4%、「衣類」が33.8%の順で高いとされています。\n\n- 国籍・地域別で最も購入率が高い費目は、韓国、台湾、香港、中国では「菓子類」が高く、米国では「その他食料品・飲料・たばこ」が高いとされています。\n\n- 費目別の購入者単価では、「宝石・貴金属」が最も高いとされています。\n\nまた、これらの購入が行われる商品の販売場所には百貨店・デパート、家電量販店、観光地の土産店、スーパーマーケットといった場所が含まれています。\n\nお土産の購入状況に関するこれらの情報が参考になるかと思います。'}

# OpenAI Tool

In [110]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

In [111]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")

In [130]:
import json
import requests
from bs4 import BeautifulSoup
from langchain.chains import TransformChain

# ニュースを検索する関数
def get_news(inputs: dict) -> dict:
    keyword = inputs["keyword"]

    # WebサイトのURLを指定
    url = f'https://news.google.com/search?q={keyword}&hl=ja&gl=JP&ceid=JP:ja'

    # Requestsを利用してWebページを取得する
    response = requests.get(url)
    html = response.text

    # BeautifulSoupを利用してWebページを解析する
    soup = BeautifulSoup(html, 'html.parser')

    # soup.selectを利用して、ニュースのタイトルを取得する
    elements = soup.select('article div div div a')

    # ニュースのトップ3を配列に取得
    news = []
    for i, element in enumerate(elements):
        news.append({"title": element.getText()})
        if i>=2:
            break

    return {"result": news}

# 関数をChain化
news_chain = TransformChain(
    input_variables=["keyword"],
    output_variables=["result"],
    transform=get_news
)


In [131]:
# 数式の文字列をもとに計算を行う関数
def calc_func(inputs: dict) -> dict:
    formula = inputs["formula"]
    answer = eval(formula)
    return {"answer": f"{answer}"}

# 関数をChain化
calc_chain = TransformChain(
    input_variables=["formula"],
    output_variables=["answer"],
    transform=calc_func
)

In [132]:
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name = "news", # 半角英数字のみ
        func=news_chain.run,
        description="ニュースを探す際に役立ちます"
    ),
    Tool(
        name = "calculation", # 半角英数字のみ
        func=calc_chain.run,
        description="計算をする場合に役立ちます"
    ),
    Tool(
        name = "Search", # 半角英数字のみ
        func=search.run,
        description="現在の出来事や世界の現状に関する質問に答える必要がある場合に役立ちます"
    ),
]

In [133]:
# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [134]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [135]:
agent_executor.invoke({"input": "LangChainとは何ですか?"})



> Entering new AgentExecutor chain...

Invoking: `Search` with `LangChain`


LangChain's flexible abstractions and extensive toolkit unlocks developers to build context-aware, reasoning LLM applications. What is LangChain? LangChain is a framework for developing applications powered by language models. It enables applications that: ... This framework consists ... LangChain is a framework for developing applications powered by language models. It enables applications that: langchain.chains ¶. Chains are easily reusable components linked together. Chains encode a sequence of calls to components like models, document retrievers, ... Building with LangChain​. LangChain enables building application that connect external sources of data and computation to LLMs. In this quickstart, we will walk ... Introduction. LangChain is a framework for developing applications powered by language models. It enables applications that: ... This framework consists of ... History edit. LangChain was launche

{'input': 'LangChainとは何ですか?',
 'output': 'LangChainは、柔軟な抽象化と豊富なツールセットによって、開発者がコンテキストに応じた推論型LLMアプリケーションを構築できるようにするフレームワークです。LangChainは、言語モデルによって動作するアプリケーションを開発するためのフレームワークであり、外部のデータソースや計算をLLMに接続するアプリケーションを構築できるようにします。LangChainは、2022年10月に機械学習スタートアップRobust Intelligenceで働いていたHarrison Chaseによってオープンソースプロジェクトとして立ち上げられました。LangChainは、製品向けのAI搭載アプリケーションを構築するための非常に人気のあるフレームワークです。'}

In [136]:
agent_executor.invoke({"input": "8掛ける12は？"})



> Entering new AgentExecutor chain...

Invoking: `calculation` with `8*12`


968 かける 12 は 96 です。

> Finished chain.


{'input': '8掛ける12は？', 'output': '8 かける 12 は 96 です。'}

In [137]:
agent_executor.invoke({"input": "夏祭りに関するニュースを教えてください"})



> Entering new AgentExecutor chain...

Invoking: `news` with `夏祭り`


[{'title': '夏祭りのお囃子の演奏で心も体も温まってもらう催し 板柳町｜NHK 青森県のニュース'}, {'title': '夏祭りで飲酒後に運転 電柱に衝突、助手席の男性死亡 危険運転致死の疑いで女を逮捕（千葉日報オンライン） - Yahoo ...'}, {'title': '【教育連携】奈良県橿原市の2023年「白橿町夏祭り」に行ってきた！花火も楽しめる町民の祭り！'}]夏祭りに関する最新のニュースをお伝えします。

1. 「夏祭りのお囃子の演奏で心も体も温まってもらう催し 板柳町｜NHK 青森県のニュース」
2. 「夏祭りで飲酒後に運転 電柱に衝突、助手席の男性死亡 危険運転致死の疑いで女を逮捕（千葉日報オンライン） - Yahoo ...」
3. 「【教育連携】奈良県橿原市の2023年「白橿町夏祭り」に行ってきた！花火も楽しめる町民の祭り！」

以上が最新の夏祭りに関するニュースです。

> Finished chain.


{'input': '夏祭りに関するニュースを教えてください',
 'output': '夏祭りに関する最新のニュースをお伝えします。\n\n1. 「夏祭りのお囃子の演奏で心も体も温まってもらう催し 板柳町｜NHK 青森県のニュース」\n2. 「夏祭りで飲酒後に運転 電柱に衝突、助手席の男性死亡 危険運転致死の疑いで女を逮捕（千葉日報オンライン） - Yahoo ...」\n3. 「【教育連携】奈良県橿原市の2023年「白橿町夏祭り」に行ってきた！花火も楽しめる町民の祭り！」\n\n以上が最新の夏祭りに関するニュースです。'}

# Toolkit:Pandas DataFrame Agent

In [148]:
import pandas as pd

df = pd.read_csv("./data2/iris.data", header=0)
df.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [144]:
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [145]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [146]:
agent.run("何行ありますか？")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape[0]'}`


150データフレームには150行あります。

> Finished chain.


'データフレームには150行あります。'

In [149]:
agent.run("Nameの基本統計量を出力してください")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Name'].describe()"}`


count             150
unique              3
top       Iris-setosa
freq               50
Name: Name, dtype: objectNameの基本統計量は以下の通りです：

- count: 150
- unique: 3
- top: Iris-setosa
- freq: 50

> Finished chain.


'Nameの基本統計量は以下の通りです：\n\n- count: 150\n- unique: 3\n- top: Iris-setosa\n- freq: 50'

In [150]:
agent.run("このデータについて説明してください")



> Entering new AgentExecutor chain...
このデータは、アイリスの花の特徴を示す4つの数値データ（がく片の長さ、がく片の幅、花びらの長さ、花びらの幅）と、アイリスの品種名（Name）からなるデータフレームです。各行は1つのアイリスの花を表し、品種名はその花の種類を示しています。このデータは、アイリスの花の特徴を分析するために使用されることがあります。

> Finished chain.


'このデータは、アイリスの花の特徴を示す4つの数値データ（がく片の長さ、がく片の幅、花びらの長さ、花びらの幅）と、アイリスの品種名（Name）からなるデータフレームです。各行は1つのアイリスの花を表し、品種名はその花の種類を示しています。このデータは、アイリスの花の特徴を分析するために使用されることがあります。'

# Toolkit:Python Agent

In [155]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool

In [156]:
tools = [PythonREPLTool()]

In [157]:
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI

In [158]:
instructions = """あなたは、質問に答えるために Python コードを作成して実行するように設計されたエージェントです。
Python コードを実行するために使用できる Python REPL にアクセスできます。
エラーが発生した場合は、コードをデバッグして再試行してください。
コードの出力は質問に答えるためにのみ使用してください。
コードを実行しなくても答えがわかる場合もありますが、答えを得るにはコードを実行する必要があります。
質問に答えるコードを書けそうにない場合は、「わかりません」と答えてください。
"""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [159]:
agent = create_openai_functions_agent(ChatOpenAI(temperature=0), tools, prompt)

In [160]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [164]:
agent_executor.invoke({"input": "100までの素数を求めよ"})



> Entering new AgentExecutor chain...

Invoking: `Python_REPL` with `def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

primes = [x for x in range(2, 101) if is_prime(x)]
primes`


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]

> Finished chain.


{'input': '100までの素数を求めよ',
 'output': '[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]'}